In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt

# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_curve, roc_auc_score

import glob
import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/data/national_nutrition/*_all.sas7bdat'

In [3]:
sorted(glob.glob(path))

['/data/national_nutrition/hn01_all.sas7bdat',
 '/data/national_nutrition/hn05_all.sas7bdat',
 '/data/national_nutrition/hn07_all.sas7bdat',
 '/data/national_nutrition/hn08_all.sas7bdat',
 '/data/national_nutrition/hn09_all.sas7bdat',
 '/data/national_nutrition/hn10_all.sas7bdat',
 '/data/national_nutrition/hn11_all.sas7bdat',
 '/data/national_nutrition/hn12_all.sas7bdat',
 '/data/national_nutrition/hn13_all.sas7bdat',
 '/data/national_nutrition/hn14_all.sas7bdat',
 '/data/national_nutrition/hn15_all.sas7bdat',
 '/data/national_nutrition/hn16_all.sas7bdat',
 '/data/national_nutrition/hn17_all.sas7bdat',
 '/data/national_nutrition/hn18_all.sas7bdat',
 '/data/national_nutrition/hn19_all.sas7bdat',
 '/data/national_nutrition/hn20_all.sas7bdat',
 '/data/national_nutrition/hn98_all.sas7bdat']

# Data Load

비타민D 대상자: (2008 ~ 2012년) 만10세이상 전수, (2013 ~ 2014년) 만10세이상 2,400명
- 비타민D: HE_vitD

In [4]:
df8 = pd.read_sas('/data/national_nutrition/hn08_all.sas7bdat')
df9 = pd.read_sas('/data/national_nutrition/hn09_all.sas7bdat')
df10 = pd.read_sas('/data/national_nutrition/hn10_all.sas7bdat')
df11 = pd.read_sas('/data/national_nutrition/hn11_all.sas7bdat')
df12 = pd.read_sas('/data/national_nutrition/hn12_all.sas7bdat')
df13 = pd.read_sas('/data/national_nutrition/hn13_all.sas7bdat')
df14 = pd.read_sas('/data/national_nutrition/hn14_all.sas7bdat')

In [5]:
common_columns = df8.columns.intersection(df9.columns).intersection(df10.columns).intersection(df11.columns).intersection(df12.columns).intersection(df13.columns).intersection(df14.columns)

In [6]:
df_temp = pd.concat([df8[common_columns], df9[common_columns]])
df_temp = pd.concat([df_temp, df10[common_columns]])
df_temp = pd.concat([df_temp, df11[common_columns]])
df_temp = pd.concat([df_temp, df12[common_columns]])
df_temp = pd.concat([df_temp, df13[common_columns]])
df = pd.concat([df_temp, df14[common_columns]])

df = df[df['age'] >= 10]
df[['HE_vitD']].isna().sum()
df = df.dropna(subset=['HE_vitD'], axis=0) # HE_vitD 결측치 제거
df = df.reset_index(drop=True)
# df

In [7]:
df

,mod_d,id,year,region,town_t,apt_t,psu,sex,age,incm,...,N_FE,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC
0,b'2022.03.08',b'A448440901',2008.0,1.0,1.0,1.0,b'A448',1.0,70.0,2.0,...,15.556678,7472.249217,4895.631231,881.735572,5273.157673,2.203960,1.561326,0.987078,16.272761,262.812776
1,b'2022.03.08',b'A448440902',2008.0,1.0,1.0,1.0,b'A448',2.0,68.0,2.0,...,11.758635,4430.090787,3680.878220,777.813158,4646.511191,2.188609,1.116872,0.729980,11.567262,206.477449
2,b'2022.03.08',b'A448680102',2008.0,1.0,1.0,1.0,b'A448',2.0,48.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b'2022.03.08',b'A448680501',2008.0,1.0,1.0,1.0,b'A448',2.0,35.0,3.0,...,18.839099,2684.503728,3310.676976,1932.710046,10725.673284,144.044389,0.909731,1.101102,9.129266,147.954071
4,b'2022.03.08',b'A448680502',2008.0,1.0,1.0,1.0,b'A448',1.0,12.0,3.0,...,6.087956,2796.580638,2984.102868,434.816014,571.273052,166.161773,1.212672,1.752693,8.201845,65.331394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,b'2022.03.08',b'P246808014',2014.0,16.0,1.0,2.0,b'P246',1.0,10.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39755,b'2022.03.08',b'P246808015',2014.0,16.0,1.0,2.0,b'P246',2.0,39.0,4.0,...,9.125954,3685.879396,1880.994894,1933.891026,1906.202132,1615.497101,2.497810,1.278473,14.143210,36.145233
39756,b'2022.03.08',b'P246811414',2014.0,16.0,1.0,2.0,b'P246',1.0,30.0,3.0,...,6.913193,2043.568097,1147.876072,300.249632,1793.774635,0.983537,0.791706,0.511638,6.020183,27.892272
39757,b'2022.03.08',b'P246811416',2014.0,16.0,1.0,2.0,b'P246',1.0,58.0,3.0,...,14.437036,1758.339159,1446.924818,189.142622,1124.184551,1.043302,1.490047,0.384598,8.750881,24.329294


# Preprocessing

## 특정컬럼 제거

- 데이터가 전부 nan인 컬럼 제거

In [8]:
l1 = []
for col in df.columns:
    if df[col].isnull().sum() == 39759:
        #print(col)
        l1.append(col)
print(l1)
print(len(l1))
df = df.drop(columns = l1)

['Y_BTH_WT', 'Y_MTM_YN', 'Y_MTM_S1', 'Y_MTM_D1', 'Y_MTM_S2', 'Y_MTM_D2', 'Y_FM_YN', 'Y_FM_S1', 'Y_FM_D1', 'Y_FM_S2', 'Y_FM_D2', 'Y_MLK_ST', 'Y_WN_ST', 'Y_SUP_YN', 'Y_SUP_KD1', 'Y_SUP_KD3', 'Y_SUP_KD4', 'Y_SUP_KD7']
18


- 쓸수 없는 컬럼 제거 

In [9]:
df_col_info = pd.read_csv('국건영(지침서)_copy.csv')

In [10]:
## 실행 주의 ## 
l2 = []
for i in df.columns[2:143]:
    if i not in list(df_col_info['variable'][:130]):
        l2.append(i)
print(l2)
df = df.drop(columns = l2)

['psu', 'kstrata', 'wt_hs', 'wt_pft', 'wt_ntr', 'wt_itvex', 'wt_tot', 'wt_pfnt', 'ID_fam', 'DE1_35', 'DC11_tp']


## object type 변수 삭제 및 변수 재배치

In [11]:
df.dtypes[df.dtypes == object].index

Index(['mod_d', 'id', 'BH1_7', 'BH2_65', 'LQ4_24', 'EC1_2_1', 'EC_wht_6',
       'BO3_11', 'BM12_2', 'N_DT_DS', 'N_DUSUAL'],
      dtype='object')

In [12]:
drop_col = list(df.dtypes[df.dtypes == object].index) + ['HE_vitD','ainc_1','ainc_unit1'] #소득변수 포함
vitD = df['HE_vitD']

In [13]:
df = df.drop(columns=drop_col)

In [14]:
df['HE_vitD'] = vitD

In [15]:
df.head()

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,2008.0,1.0,1.0,1.0,1.0,70.0,2.0,1.0,2.0,1.0,...,7472.249217,4895.631231,881.735572,5273.157673,2.203960,1.561326,0.987078,16.272761,262.812776,14.85
1,2008.0,1.0,1.0,1.0,2.0,68.0,2.0,1.0,2.0,1.0,...,4430.090787,3680.878220,777.813158,4646.511191,2.188609,1.116872,0.729980,11.567262,206.477449,14.05
2,2008.0,1.0,1.0,1.0,2.0,48.0,4.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.36
3,2008.0,1.0,1.0,1.0,2.0,35.0,3.0,3.0,3.0,4.0,...,2684.503728,3310.676976,1932.710046,10725.673284,144.044389,0.909731,1.101102,9.129266,147.954071,18.52
4,2008.0,1.0,1.0,1.0,1.0,12.0,3.0,3.0,4.0,4.0,...,2796.580638,2984.102868,434.816014,571.273052,166.161773,1.212672,1.752693,8.201845,65.331394,18.47


## 비해당, 모름(8,88,888,,) 구분

In [16]:
df_col_info

,variable,variable description,option description,data type
0,year,조사연도,2008 | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 |,numeric
1,region,16개 시도,1 : 서울 | 2 : 부산 | 3 : 대구 | 4 : 인천 | 5 : 광주 | 6...,category
2,town_t,동/읍면 구분,1 : 동 | 2 : 읍·면 |,category
3,apt_t,아파트 구분,1 : 일반 | 2 : 아파트 |,category
4,sex,성별,1 : 남자 | 2 : 여자 |,category
...,...,...,...,...
403,N_RETIN,레티놀,NaN,numeric
404,N_B1,티아민,NaN,numeric
405,N_B2,리보플라빈,NaN,numeric
406,N_NIAC,나이아신,NaN,numeric


In [17]:
df_col_info = df_col_info[df_col_info['variable'].map(lambda x: x not in l1)] # 유아 관련 nan값
df_col_info = df_col_info[df_col_info['variable'].map(lambda x: x not in drop_col)] # 위에서 제거한 열
df_col_info

,variable,variable description,option description,data type
0,year,조사연도,2008 | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 |,numeric
1,region,16개 시도,1 : 서울 | 2 : 부산 | 3 : 대구 | 4 : 인천 | 5 : 광주 | 6...,category
2,town_t,동/읍면 구분,1 : 동 | 2 : 읍·면 |,category
3,apt_t,아파트 구분,1 : 일반 | 2 : 아파트 |,category
4,sex,성별,1 : 남자 | 2 : 여자 |,category
...,...,...,...,...
403,N_RETIN,레티놀,NaN,numeric
404,N_B1,티아민,NaN,numeric
405,N_B2,리보플라빈,NaN,numeric
406,N_NIAC,나이아신,NaN,numeric


### 기준 다른 변수 처리

In [18]:
# 소득금액 기준 1:연 2:월

# 2008-2009
ind_07_09 = df['ainc'][(df['year']==2007.0) & (df['year']==2009.0)].index


# 2010-2012
ind_10_12 = df['ainc'][(df['year']>=2010.0) & (df['year']<=2012.0)].index


# 2013-2014
ind_13_14 = df['ainc'][(df['year']==2013.0) | (df['year']==2014.0)].index


        
for ind in ind_10_12: # 연
    if (df['ainc'].iloc[ind] > 1) & (df['ainc'].iloc[ind] <= 17):
        df['ainc'].iloc[ind] = 17
    elif df['ainc'].iloc[ind] >= 900:
        df['ainc'].iloc[ind] = 900
    else:
        pass
        
for ind in ind_13_14: # 월
    if (df['ainc'].iloc[ind] >= 900):
        df['ainc'].iloc[ind] = 900

In [19]:
lst_2_0 = ['BM12', 'OR1_2'] # 치과진료 미검진 여부 , 1년간 구강검진 여부
for col in df.columns:
    if col in lst_2_0:
        df.loc[df[col] == 2, col] = 0
df['BM12'].value_counts()
df['OR1_2'].value_counts()

0.0    27754
1.0    10989
9.0      389
Name: OR1_2, dtype: int64

### 범주형 변수

In [20]:
df_col_cat = df_col_info[df_col_info['data type']=='category']
df_col_cat['data type'].unique()

array(['category'], dtype=object)

In [21]:
df_col_cat

,variable,variable description,option description,data type
1,region,16개 시도,1 : 서울 | 2 : 부산 | 3 : 대구 | 4 : 인천 | 5 : 광주 | 6...,category
2,town_t,동/읍면 구분,1 : 동 | 2 : 읍·면 |,category
3,apt_t,아파트 구분,1 : 일반 | 2 : 아파트 |,category
4,sex,성별,1 : 남자 | 2 : 여자 |,category
6,incm,소득 사분위수(개인),1 : 하 | 2 : 중하 | 3 : 중상 | 4 : 상 |,category
...,...,...,...,...
364,LK_EDU,영양교육 여부,1:예|2:아니오|9:모름/무응답,category
365,LF_CARE,식생활지원프로그램 수혜경험,1:예|2:아니오|9:모름/무응답,category
366,LF_SAFE,식생활형편,1: 충분한 양과 다양한 음식 먹을 수 있었다|2:충분한 양의 음식을 먹을 수 있었...,category
386,N_DIET,식이요법여부,1:예|2:아니오|9:모름/무응답,category


In [22]:
df_col_cat_list = list(df_col_cat['variable'])

df_f2s = df[df_col_cat_list].copy()
df_f2s

,region,town_t,apt_t,sex,incm,ho_incm,incm5,ho_incm5,edu,occp,...,O_DFID,O_DMFIP,L_OUT_FQ,LK_LB_IT,LK_LB_EF,LK_EDU,LF_CARE,LF_SAFE,N_DIET,N_DIET_WHY
0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,...,0.0,1.0,3.0,88.0,8.0,2.0,2.0,2.0,2.0,8.0
1,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,7.0,...,0.0,1.0,3.0,88.0,8.0,2.0,2.0,2.0,1.0,2.0
2,1.0,1.0,1.0,2.0,4.0,4.0,5.0,5.0,3.0,7.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,2.0,3.0,3.0,3.0,4.0,3.0,7.0,...,0.0,1.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,8.0
4,1.0,1.0,1.0,1.0,3.0,3.0,4.0,4.0,1.0,NaN,...,0.0,0.0,3.0,88.0,8.0,1.0,2.0,1.0,2.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,16.0,1.0,2.0,1.0,4.0,4.0,5.0,5.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39755,16.0,1.0,2.0,2.0,4.0,4.0,5.0,5.0,4.0,7.0,...,NaN,NaN,3.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0
39756,16.0,1.0,2.0,1.0,3.0,3.0,3.0,4.0,4.0,3.0,...,NaN,NaN,3.0,88.0,8.0,2.0,2.0,2.0,1.0,2.0
39757,16.0,1.0,2.0,1.0,3.0,3.0,3.0,4.0,3.0,6.0,...,NaN,NaN,6.0,8.0,1.0,2.0,2.0,2.0,2.0,8.0


In [23]:
tmp = []
for i in df_col_cat.index:
    if df_col_cat.loc[i]['variable'] not in df.columns:
        tmp.append(i)
df_col_cat.drop(tmp,axis = 0,inplace = True)

In [24]:
df_col_cat.head()

,variable,variable description,option description,data type
1,region,16개 시도,1 : 서울 | 2 : 부산 | 3 : 대구 | 4 : 인천 | 5 : 광주 | 6...,category
2,town_t,동/읍면 구분,1 : 동 | 2 : 읍·면 |,category
3,apt_t,아파트 구분,1 : 일반 | 2 : 아파트 |,category
4,sex,성별,1 : 남자 | 2 : 여자 |,category
6,incm,소득 사분위수(개인),1 : 하 | 2 : 중하 | 3 : 중상 | 4 : 상 |,category


`df_f2s` : `df` 의 범주형 변수의 값들을 문자형으로 변경한 데이터

`dic_float2str` : 범주형 변수의 값을 문자형으로 변경할 때 사용한 딕셔너리

In [25]:
# float to string
f2s_dic_list= {}
for i in range(len(df_col_cat)):
    
        val_name = df_col_cat.iloc[i][0]          # 변수명
        option = df_col_cat.iloc[i][2]            # option 값들, ex)1 : 일반 | 2 : 아파트 |
        
        if option[-1] == '|':
            option = option[:-1]
        
        option = option.split('|')
        dic_float2str = {}
        
        for i in option:
            print(i)
            k,v = i.split(':')
            dic_float2str[float(k.strip())] = v.strip()
            
        f2s_dic_list[val_name] = dic_float2str
        df_f2s[val_name] = df_f2s[val_name].map(dic_float2str)

1 : 서울 
 2 : 부산 
 3 : 대구 
 4 : 인천 
 5 : 광주 
 6 : 대전 
 7 : 울산 
 8 : 세종 
 9 : 경기 
 10 : 강원 
 11 : 충북 
 12 : 충남 
 13 : 전북 
 14 : 전남 
 15 : 경북 
 16 : 경남 
 17 : 제주 
1 : 동 
 2 : 읍·면 
1 : 일반 
 2 : 아파트 
1 : 남자 
 2 : 여자 
1 : 하 
 2 : 중하 
 3 : 중상 
 4 : 상 
1 : 하 
 2 : 중하 
 3 : 중상 
 4 : 상 
1 : 하 
 2 : 중하 
 3 : 중 
 4 : 중상 
 5 : 상 
1 : 하 
 2 : 중하 
 3 : 중 
 4 : 중상 
 5 : 상 
1 : 초졸이하 
 2 : 중졸 
 3 : 고졸 
 4 : 대졸이상 
1 : 관리자, 전문가 및 관련 종사자 
 2 : 사무종사자 
 3 : 서비스 및 판매 종사자 
 4 : 농림어업 숙련 종사자 
 5 : 기능원, 장치?기계조작 및 조립종사자 
 6 : 단순노무종사자 
 7 : 무직(주부, 학생 등) 
1 : 1명 
 2 : 2명 
 3 : 3명 
 4 : 4명 
 5 : 5명 
 6 : 6명 이상 
 9 : 모름, 무응답 
1 : 1세대 가구 - 1인가구 
 2 : 1세대 가구 - 부부 
 3 : 1세대 가구 - 기타 
 4 : 2세대 가구 - 부부+미혼자녀 
 5 : 2세대 가구 ? 편부모+미혼자녀 
 6 : 2세대 가구 ? 기타 
 7 : 3세대 이상 가구 
 9 : 모름, 무응답 
10 : 수급경험 있음(과거 또는 현재) 
 20 : 수급경험 없음 
 99 : 모름, 무응답 
1 : 아니오 
 2 : 1채 있음 
 3 : 2채 이상 있음 
 9 : 모름, 무응답 
1 : 단독주택 
 2 : 아파트 
 3 : 연립주택 
 4 : 다세대 주택 
 5 : 기타 
 9 : 모름, 무응답 
1 : 기혼 
 2 : 미혼 
 9 : 모름, 무응답 
1 : 유배우자, 동거 
 2 : 유배우자, 별거  
 3 : 사별  
 4 : 이혼

In [26]:
df_f2s.head()

,region,town_t,apt_t,sex,incm,ho_incm,incm5,ho_incm5,edu,occp,...,O_DFID,O_DMFIP,L_OUT_FQ,LK_LB_IT,LK_LB_EF,LK_EDU,LF_CARE,LF_SAFE,N_DIET,N_DIET_WHY
0,서울,동,일반,남자,중하,하,중하,하,초졸이하,NaN,...,우식경험유치 수=0,우식경험유치 수 >=1,주1~6회,비해당,비해당,아니오,아니오,충분한 양의 음식을 먹을 수 있었으나. 다양한 음식은 먹지 못했다,아니오,비해당
1,서울,동,일반,여자,중하,하,중하,하,초졸이하,"무직(주부, 학생 등)",...,우식경험유치 수=0,우식경험유치 수 >=1,주1~6회,비해당,비해당,아니오,아니오,충분한 양의 음식을 먹을 수 있었으나. 다양한 음식은 먹지 못했다,예,체중을 조절하기 위해서
2,서울,동,일반,여자,상,상,상,상,고졸,"무직(주부, 학생 등)",...,우식경험유치 수=0,우식경험유치 수=0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,서울,동,일반,여자,중상,중상,중,중상,고졸,"무직(주부, 학생 등)",...,우식경험유치 수=0,우식경험유치 수 >=1,주1~6회,열량,아니오,아니오,아니오,충분한 양과 다양한 음식 먹을 수 있었다,아니오,비해당
4,서울,동,일반,남자,중상,중상,중상,중상,초졸이하,NaN,...,우식경험유치 수=0,우식경험유치 수=0,주1~6회,비해당,비해당,예,아니오,충분한 양과 다양한 음식 먹을 수 있었다,아니오,비해당


In [27]:
f2s_dic_list

{'region': {1.0: '서울',
  2.0: '부산',
  3.0: '대구',
  4.0: '인천',
  5.0: '광주',
  6.0: '대전',
  7.0: '울산',
  8.0: '세종',
  9.0: '경기',
  10.0: '강원',
  11.0: '충북',
  12.0: '충남',
  13.0: '전북',
  14.0: '전남',
  15.0: '경북',
  16.0: '경남',
  17.0: '제주'},
 'town_t': {1.0: '동', 2.0: '읍·면'},
 'apt_t': {1.0: '일반', 2.0: '아파트'},
 'sex': {1.0: '남자', 2.0: '여자'},
 'incm': {1.0: '하', 2.0: '중하', 3.0: '중상', 4.0: '상'},
 'ho_incm': {1.0: '하', 2.0: '중하', 3.0: '중상', 4.0: '상'},
 'incm5': {1.0: '하', 2.0: '중하', 3.0: '중', 4.0: '중상', 5.0: '상'},
 'ho_incm5': {1.0: '하', 2.0: '중하', 3.0: '중', 4.0: '중상', 5.0: '상'},
 'edu': {1.0: '초졸이하', 2.0: '중졸', 3.0: '고졸', 4.0: '대졸이상'},
 'occp': {1.0: '관리자, 전문가 및 관련 종사자',
  2.0: '사무종사자',
  3.0: '서비스 및 판매 종사자',
  4.0: '농림어업 숙련 종사자',
  5.0: '기능원, 장치?기계조작 및 조립종사자',
  6.0: '단순노무종사자',
  7.0: '무직(주부, 학생 등)'},
 'cfam': {1.0: '1명',
  2.0: '2명',
  3.0: '3명',
  4.0: '4명',
  5.0: '5명',
  6.0: '6명 이상',
  9.0: '모름, 무응답'},
 'genertn': {1.0: '1세대 가구 - 1인가구',
  2.0: '1세대 가구 - 부부',
  3.0: '1세대 가구 - 기타',
  4

In [29]:
# 모름 값(999)를 결측치로 보는 경우
df_s2f = df_f2s.copy()
s2f_dic_list= {}

for i in range(len(df_col_cat)):
        val_name = df_col_cat.iloc[i][0]
        option = df_col_cat.iloc[i][2]
        
        if option[-1] == '|':
            option = option[:-1]
        option = option.split('|')
        dic_str2float = {}
        tmp = 1.0
        
        for i in option:
            k,v = i.split(':')
            #print(v.strip()=='모름')
            #print('비해당' in v.strip())
            for s in ['비해당','다니지 않음','전혀 하지 않음']:
                if s in v.strip():
                    dic_str2float[v.strip()] = float(-1)
                    break
            for s in ['모름','응답거부','무응답','판정불능']:
                if s in v.strip():
                    dic_str2float[v.strip()] = np.nan
                    break
            if  dic_str2float.get(v.strip(),'0') == '0':
                dic_str2float[v.strip()] = tmp
                tmp += float(1)
        
        s2f_dic_list[val_name] = dic_str2float

        #dic1
        df_s2f[val_name] = df_s2f[val_name].map(dic_str2float)
        df_s2f[val_name].unique()

In [30]:
df_s2f.head()

,region,town_t,apt_t,sex,incm,ho_incm,incm5,ho_incm5,edu,occp,...,O_DFID,O_DMFIP,L_OUT_FQ,LK_LB_IT,LK_LB_EF,LK_EDU,LF_CARE,LF_SAFE,N_DIET,N_DIET_WHY
0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,...,1.0,2.0,3.0,-1.0,-1.0,2.0,2.0,2.0,2.0,-1.0
1,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,7.0,...,1.0,2.0,3.0,-1.0,-1.0,2.0,2.0,2.0,1.0,2.0
2,1.0,1.0,1.0,2.0,4.0,4.0,5.0,5.0,3.0,7.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,2.0,3.0,3.0,3.0,4.0,3.0,7.0,...,1.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,-1.0
4,1.0,1.0,1.0,1.0,3.0,3.0,4.0,4.0,1.0,NaN,...,1.0,1.0,3.0,-1.0,-1.0,1.0,2.0,1.0,2.0,-1.0


In [31]:
s2f_dic_list

{'region': {'서울': 1.0,
  '부산': 2.0,
  '대구': 3.0,
  '인천': 4.0,
  '광주': 5.0,
  '대전': 6.0,
  '울산': 7.0,
  '세종': 8.0,
  '경기': 9.0,
  '강원': 10.0,
  '충북': 11.0,
  '충남': 12.0,
  '전북': 13.0,
  '전남': 14.0,
  '경북': 15.0,
  '경남': 16.0,
  '제주': 17.0},
 'town_t': {'동': 1.0, '읍·면': 2.0},
 'apt_t': {'일반': 1.0, '아파트': 2.0},
 'sex': {'남자': 1.0, '여자': 2.0},
 'incm': {'하': 1.0, '중하': 2.0, '중상': 3.0, '상': 4.0},
 'ho_incm': {'하': 1.0, '중하': 2.0, '중상': 3.0, '상': 4.0},
 'incm5': {'하': 1.0, '중하': 2.0, '중': 3.0, '중상': 4.0, '상': 5.0},
 'ho_incm5': {'하': 1.0, '중하': 2.0, '중': 3.0, '중상': 4.0, '상': 5.0},
 'edu': {'초졸이하': 1.0, '중졸': 2.0, '고졸': 3.0, '대졸이상': 4.0},
 'occp': {'관리자, 전문가 및 관련 종사자': 1.0,
  '사무종사자': 2.0,
  '서비스 및 판매 종사자': 3.0,
  '농림어업 숙련 종사자': 4.0,
  '기능원, 장치?기계조작 및 조립종사자': 5.0,
  '단순노무종사자': 6.0,
  '무직(주부, 학생 등)': 7.0},
 'cfam': {'1명': 1.0,
  '2명': 2.0,
  '3명': 3.0,
  '4명': 4.0,
  '5명': 5.0,
  '6명 이상': 6.0,
  '모름, 무응답': nan},
 'genertn': {'1세대 가구 - 1인가구': 1.0,
  '1세대 가구 - 부부': 2.0,
  '1세대 가구 - 기타': 3.0,
  '

In [32]:
#df = df_s2f

### 범주 + 연속형 변수

In [33]:
df_col_both = df_col_info[df_col_info['data type']=='numeric_age']
df_col_both['data type'].unique()

array(['numeric_age'], dtype=object)

In [34]:
df_col_both = df_col_both.reset_index(drop=True)
df_col_both = df_col_both.iloc[:49]      # 유아 데이터 삭제
df_col_both

,variable,variable description,option description,data type
0,age,만나이,1~79 : 1~79세 | 80 : 80세이상 |,numeric_age
1,ainc,월평균 가구총소득,17 : 월 17만원 이하 | 17초과 ~ 1500미만 : (연속형 소득금액) | ...,numeric_age
2,D_2_wk,최근2주간 불편감일수,"88 : 비해당(문항2-②) | 99 : 모름, 무응답 |",numeric_age
3,DI1_ag,고혈압 현재 유병 여부,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
4,DI2_ag,이상지질혈증 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
5,DI3_ag,뇌졸중 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
6,DI5_ag,심근경색증 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
7,DI6_ag,협심증 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
8,DM2_ag,골관절염 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
9,DM3_ag,류마티스성 관절염 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age


In [35]:
age_var_ind = []          # 나이 관련 변수 인덱스
n_age_var_ind = []        # 나이와 상관 없는 변수 인덱스
for i in range(df_col_both.shape[0]):
    var_name = df_col_both.iloc[i]['variable']
    if 'ag' in var_name or 'ya' in var_name:
        age_var_ind.append(i)
    else:
        n_age_var_ind.append(i)
age_var_ind.append(34)         # 'BD2' 추가

In [36]:
df_agevar_info = df_col_both.iloc[age_var_ind].reset_index(drop=True)
df_n_agevar_info = df_col_both.iloc[n_age_var_ind].reset_index(drop=True)
# df_agevar_info

#### 나이 관련 변수 ~ 결측치 개수 확인

In [37]:
df_agevar_info

,variable,variable description,option description,data type
0,age,만나이,1~79 : 1~79세 | 80 : 80세이상 |,numeric_age
1,DI1_ag,고혈압 현재 유병 여부,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
2,DI2_ag,이상지질혈증 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
3,DI3_ag,뇌졸중 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
4,DI5_ag,심근경색증 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
5,DI6_ag,협심증 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
6,DM2_ag,골관절염 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
7,DM3_ag,류마티스성 관절염 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
8,DJ2_ag,폐결핵 진단시기,"0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(청소년, 소아...",numeric_age
9,DJ4_ag,"천식 진단시기 (소아, 청소년 포함)",0~79 : 0~79세 | 80 : 80세이상 | 888 : 비해당(의사진단 받지...,numeric_age


In [38]:
agevar_list = list(df_agevar_info['variable'])

df_agevar = df[agevar_list].copy()    # df_agevar : 전체 데이터 중 나이관련 변수
df_agevar

,age,DI1_ag,DI2_ag,DI3_ag,DI5_ag,DI6_ag,DM2_ag,DM3_ag,DJ2_ag,DJ4_ag,...,DC6_ag,DC11_ag,DK8_ag,DK9_ag,DK4_ag,DI9_ya,DF1_ya,DN6_ya,DJ9_ya,BA2_2_2
0,70.0,888.0,65.0,888.0,888.0,888.0,69.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
1,68.0,888.0,64.0,888.0,888.0,888.0,60.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
2,48.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
3,35.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
4,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,888.0,...,NaN,NaN,NaN,NaN,NaN,888.0,888.0,888.0,888.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,10.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,2.0,888.0
39755,39.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0
39756,30.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0
39757,58.0,50.0,55.0,888.0,888.0,888.0,57.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0


In [39]:
# df_agevar['DI1_ag'].value_counts()[[888,999]].sum()
df_agevar['age'].isnull().sum()

0

In [41]:
def missing_val_count(target,col):
    if target == 'nan':
        return df_agevar[col].isnull().sum()
    else:
        return df_agevar[col].value_counts()[target].sum() if target in df_agevar[col].unique() else 0

In [42]:
missing_val_count('nan','DI1_ag')

1474

In [43]:
missing_val_count(888,'DI1_ag')

31063

In [44]:
df_agevar_mv_count = pd.DataFrame(index = list(df_agevar.columns),columns= ['nan',888,999])
for i,col in enumerate(df_agevar.columns):
    for target in ['nan',888,999]:
        df_agevar_mv_count.loc[col][target] = missing_val_count(target,col)
        #print(target)
df_agevar_mv_count

,nan,888,999
age,0,0,0
DI1_ag,1474,31063,289
DI2_ag,1474,34786,285
DI3_ag,1474,37389,278
DI5_ag,1474,37757,278
DI6_ag,1474,37457,281
DM2_ag,1474,34262,297
DM3_ag,1474,37321,282
DJ2_ag,1474,36387,286
DJ4_ag,531,37626,291


- 888을 결측치로 볼 경우 : 888값을 가지고 있는 데이터가 많아서 대부분의 데이터가 결측치로 인식되기때문에 888을 결측치로 처리하지 않음.

- 따라서 888을 결측치로 보고 삭제시킬 수 없기때문에 1번 방법(범주형 데이터 모두 삭제)는 불가능함

In [45]:
df_agevar_mv_count['ratio(nan+999)'] = (df_agevar_mv_count['nan'] + df_agevar_mv_count[999])/len(df)
df_agevar_mv_count['실제 결측치 비율'] = (df_agevar_mv_count['nan'] + df_agevar_mv_count[999])/(len(df)- df_agevar_mv_count[888])
df_agevar_mv_count

,nan,888,999,ratio(nan+999),실제 결측치 비율
age,0,0,0,0.0,0.0
DI1_ag,1474,31063,289,0.044342,0.202737
DI2_ag,1474,34786,285,0.044242,0.35371
DI3_ag,1474,37389,278,0.044065,0.739241
DI5_ag,1474,37757,278,0.044065,0.875125
DI6_ag,1474,37457,281,0.044141,0.762381
DM2_ag,1474,34262,297,0.044543,0.322176
DM3_ag,1474,37321,282,0.044166,0.720263
DJ2_ag,1474,36387,286,0.044267,0.521945
DJ4_ag,531,37626,291,0.020675,0.385373


#### 나이 관련 변수 ~ 결측치 처리

**처리 방법**

`888` : 비해당인 카테고리로 분류
`999` : nan값으로 분류

888과 nan(999+nan) 을 제외한 연속형 변수를 나이대 별로 (10대, 20대 등)으로 범주화 한 뒤 최빈값으로 결측치를 처리함.



In [46]:
def age_category(x):
    if (x<=19):
        return 1
    elif (x>=20) and (x<=29):
        return 2
    elif (x>=30) and (x<=39):
        return 3
    elif (x>=40) and (x<=49):
        return 4
    elif (x>=50) and (x<=59):
        return 5
    elif (x>=60) and (x<=69):
        return 6
    elif (x>=70) and (x<=79):
        return 7
    elif (x>=80) and (x<888):
        return 8
#     elif x == 888:
#         return 888
#     else:
#         return 999

In [47]:
df_agevar['DI1_ag'].map(age_category).value_counts()

5.0    2192
6.0    1949
4.0    1333
7.0     781
3.0     451
2.0     120
8.0      75
1.0      32
Name: DI1_ag, dtype: int64

In [48]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

encoder_dict = {}
for col in list(df_agevar_info['variable']):
    df_agevar.loc[df_agevar[col]==999,col] = np.nan
    tmp_df = df_agevar[df_agevar[col]!=888][col]
    tmp_df = tmp_df.map(age_category).to_frame()
    freq_imputer = SimpleImputer(strategy='most_frequent')
    
    tmp_df[col] = freq_imputer.fit_transform(tmp_df[[col]])
    df_agevar.loc[df_agevar[col]!=888,col] = tmp_df[col]

    # 라벨인코더
    le = LabelEncoder()
    df_agevar[col] = le.fit_transform(df_agevar[[col]])
    encoder_dict[col] = le.classes_

In [49]:
df_agevar['DI1_ag'].value_counts()

8    31063
4     3955
5     1949
3     1333
6      781
2      451
1      120
7       75
0       32
Name: DI1_ag, dtype: int64

아래와 같은 경우라면 0 : 10대, 1 : 20대, 2 : 30대, 3 : 40대, 4 : 50대 ... 8 : 888( 비해당 )

In [50]:
encoder_dict['DI1_ag']

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8., 888.])

In [51]:
df_agevar.head()

,age,DI1_ag,DI2_ag,DI3_ag,DI5_ag,DI6_ag,DM2_ag,DM3_ag,DJ2_ag,DJ4_ag,...,DC6_ag,DC11_ag,DK8_ag,DK9_ag,DK4_ag,DI9_ya,DF1_ya,DN6_ya,DJ9_ya,BA2_2_2
0,6,8,5,8,7,8,5,8,8,8,...,5,8,7,7,6,0,0,0,0,8
1,5,8,5,8,7,8,5,8,8,8,...,5,8,7,7,6,0,0,0,0,8
2,3,8,8,8,7,8,8,8,8,8,...,5,8,7,7,6,0,0,0,0,8
3,2,8,8,8,7,8,8,8,8,8,...,5,8,7,7,6,0,0,0,0,8
4,0,4,4,5,4,5,4,4,1,8,...,2,4,1,4,2,1,1,1,1,0


#### 나이와 관련 없는 변수 ~ 결측치 개수 확인, 결측치 처리
**처리방법**


범주 + 연속형 변수 중 나이와 관련 없는 변수에서의 '비해당(88,888 등)' 은 0이라고 볼 수 있음

ex) 1달간 최근 결근 일수 라는 변수의 비해당은 결근을 한 적이 없음을 나타내며 따라서 0이라고 볼 수 있음

`888` : 0 , `999` : nan 으로 처리한다면 이 변수들은 모두 연속형 변수라고 볼 수 있다.

따라서 결측치를 중앙값으로 처리함

In [52]:
# 누락된 정보 추가
df_n_agevar_info.loc[df_n_agevar_info['variable']=='BS6_2_2','option description'] = '88:비해당 | 99:모름'
df_n_agevar_info.loc[df_n_agevar_info['variable']=='BS6_2_1','option description'] = '88:비해당 | 99:모름'
df_n_agevar_info.loc[df_n_agevar_info['variable']=='BS6_2','option description'] = '888:비해당 | 999:모름| 8888 : 비해당 | 9999 : 모름, 무응답'
df_n_agevar_info.loc[df_n_agevar_info['variable']=='BS2','option description'] = '88:비해당 | 99:모름 | 888:비해당 | 999:모름,무응답'

df_n_agevar_info

,variable,variable description,option description,data type
0,ainc,월평균 가구총소득,17 : 월 17만원 이하 | 17초과 ~ 1500미만 : (연속형 소득금액) | ...,numeric_age
1,D_2_wk,최근2주간 불편감일수,"88 : 비해당(문항2-②) | 99 : 모름, 무응답 |",numeric_age
2,LQ1_mn,최근 1달간 와병일수,"88 : 비해당 | 99 : 모름, 무응답",numeric_age
3,LQ2_mn,최근 1달간 결근일수,"88 : 비해당 | 99 : 모름, 무응답",numeric_age
4,BD2,(만12세이상) 음주 시작 연령,1~79 : 1~79세 | 80 : 80세이상 | 888 : 비해당 | 999 :...,numeric_age
5,BA2_2_2,(성인) 1년간 음주운전 횟수: 자전거,"888 : 비해당 | 999 : 모름, 무응답",numeric_age
6,BA2_2_4,(성인) 1년간 음주운전 횟수: 오토바이,"888 : 비해당 | 999 : 모름, 무응답",numeric_age
7,BA2_2_6,(성인) 1년간 음주운전 횟수: 자동차,"888 : 비해당 | 999 : 모름, 무응답",numeric_age
8,BP8,하루 평균 수면시간,"88 : 비해당 | 99 : 모름, 무응답",numeric_age
9,BS3_2,(성인) 현재흡연자 하루평균 흡연량,"888 : 비해당 | 999 : 모름, 무응답",numeric_age


In [53]:
df_n_agevar_info = df_n_agevar_info.drop(index = 5).reset_index(drop=True)
df_n_agevar_info

,variable,variable description,option description,data type
0,ainc,월평균 가구총소득,17 : 월 17만원 이하 | 17초과 ~ 1500미만 : (연속형 소득금액) | ...,numeric_age
1,D_2_wk,최근2주간 불편감일수,"88 : 비해당(문항2-②) | 99 : 모름, 무응답 |",numeric_age
2,LQ1_mn,최근 1달간 와병일수,"88 : 비해당 | 99 : 모름, 무응답",numeric_age
3,LQ2_mn,최근 1달간 결근일수,"88 : 비해당 | 99 : 모름, 무응답",numeric_age
4,BD2,(만12세이상) 음주 시작 연령,1~79 : 1~79세 | 80 : 80세이상 | 888 : 비해당 | 999 :...,numeric_age
5,BA2_2_4,(성인) 1년간 음주운전 횟수: 오토바이,"888 : 비해당 | 999 : 모름, 무응답",numeric_age
6,BA2_2_6,(성인) 1년간 음주운전 횟수: 자동차,"888 : 비해당 | 999 : 모름, 무응답",numeric_age
7,BP8,하루 평균 수면시간,"88 : 비해당 | 99 : 모름, 무응답",numeric_age
8,BS3_2,(성인) 현재흡연자 하루평균 흡연량,"888 : 비해당 | 999 : 모름, 무응답",numeric_age
9,BS6_2,(성인) 과거흡연자 흡연기간(월 환산),"888:비해당 | 999:모름| 8888 : 비해당 | 9999 : 모름, 무응답",numeric_age


In [54]:
n_agevar_list = list(df_n_agevar_info['variable'])

df_n_agevar = df[n_agevar_list].copy()    # df_agevar : 전체 데이터 중 나이관련 변수
df_n_agevar

,ainc,D_2_wk,LQ1_mn,LQ2_mn,BD2,BA2_2_4,BA2_2_6,BP8,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,40.000000,88.0,88.0,88.0,14.0,888.0,888.0,9.0,10.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,1.0,0.0
1,40.000000,14.0,88.0,88.0,888.0,888.0,888.0,7.0,888.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,0.0,30.0
2,700.000000,88.0,88.0,88.0,888.0,888.0,888.0,9.0,888.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,88.0,88.0
3,333.333333,14.0,88.0,88.0,18.0,888.0,888.0,6.0,888.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,0.0,20.0
4,333.333333,88.0,88.0,88.0,888.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,88.0,88.0,888.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,666.666667,5.0,88.0,88.0,888.0,888.0,888.0,88.0,888.0,8888.0,88.0,88.0,888.0,888.0,88.0,888.0,88.0,88.0
39755,666.666667,2.0,88.0,88.0,22.0,888.0,888.0,6.0,888.0,8888.0,88.0,88.0,888.0,888.0,88.0,888.0,0.0,20.0
39756,430.000000,88.0,88.0,88.0,19.0,888.0,888.0,6.0,888.0,8888.0,88.0,88.0,888.0,888.0,88.0,888.0,1.0,30.0
39757,430.000000,7.0,88.0,88.0,20.0,888.0,888.0,6.0,888.0,180.0,15.0,0.0,20.0,888.0,88.0,888.0,4.0,0.0


In [55]:
df_n_agevar['D_2_wk'].value_counts()[88]

30336

In [56]:
def missing_val_count_2(target,col):
    if target == 'nan':
        return df_n_agevar[col].isnull().sum()
    else:
        return df_n_agevar[col].value_counts()[target].sum() if target in df_n_agevar[col].unique() else 0

In [57]:
df_n_agevar_mv_count = pd.DataFrame(index = list(df_n_agevar.columns),columns= ['nan','비해당','모름'])
for col in df_n_agevar.columns:
    df_n_agevar_mv_count.loc[col,'nan'] = df_n_agevar[col].isnull().sum()
dict_opts = {}
for i,col in enumerate(df_n_agevar.columns):
    #print(col)
    dict_opt = {}
    for opt in df_n_agevar_info[df_n_agevar_info['variable'] == col]['option description']:
        if opt[-1] == '|':
            opt = opt[:-1]

        for j in opt.split('|'):
            j = j.strip()
            #print(col,j)
            if j.split(':')[1].strip() in ['비해당','비해당(문항2-②)','전혀 하지 않음']:
#                 print(col,j.split(':')[0])
                dict_opt['0'] = j.split(':')[1].strip()
                #print(j.split(':')[0])
                df_n_agevar_mv_count.loc[col]['비해당'] = missing_val_count_2(int(j.split(':')[0]), col)
                df_n_agevar.loc[df_n_agevar[col]==int(j.split(':')[0]),col] = 0
                
            if j.split(':')[1].strip() in ['모름, 무응답','모름','모름,무응답']:
#                 print(col,j.split(':')[0])
                dict_opt[j.split(':')[0]] = j.split(':')[1].strip()
                df_n_agevar_mv_count.loc[col]['모름'] = missing_val_count_2(int(j.split(':')[0]), col)
                df_n_agevar.loc[df_n_agevar[col]==int(j.split(':')[0]),col] = np.nan
    dict_opts[col] = dict_opt
        
    
#     for target in ['nan',888,999]:
#         df_agevar_mv_count.loc[col][target] = missing_val_count(target,col)
#         #print(target)
# df_n_agevar_mv_count

In [58]:
dict_opts['BS6_2']

{'0': '비해당', '999': '모름', '9999 ': '모름, 무응답'}

In [59]:
df_n_agevar['BS6_2'].unique()

array([  0.,  nan, 210., 240.,  72., 218., 180.,   1.,  60., 288., 300.,
        36., 444., 144., 222., 156., 480., 408., 120.,  96.,  24., 276.,
       264., 106., 216.,  32., 468., 154., 132.,  97., 420., 204., 732.,
        48., 360., 108., 372., 540., 336.,  94., 576., 130., 348.,  12.,
       123., 456., 170.,   2., 250.,  18.,  10.,   3., 138.,   6., 100.,
       324., 228., 720., 192., 564., 168., 504., 247., 242., 600., 612.,
       312.,  13., 696., 492.,  70., 362., 527.,   5.,  30., 384.,  16.,
        84., 432., 135., 134., 182., 708., 528.,  42., 792., 524.,  40.,
       744., 366.,  11., 102., 191., 173.,  99., 146.,   7., 185., 601.,
       660.,  89., 190., 396., 143., 592., 571., 214.,  76., 202., 756.,
       425., 252.,  90.,  65., 516.,  29.,  82.,  14., 186., 518., 552.,
        15.,  56., 162., 116., 166.,  85., 101., 630., 378.,  92., 315.,
       452.,  55., 305.,  41., 306.,  87., 241., 158., 217., 636., 284.,
       253., 126.,  22., 316.,  54., 245., 282., 43

In [60]:
df_n_agevar['BP8'].unique()

array([ 9. ,  7. ,  6. ,  nan,  4. , 10. ,  8. ,  5. ,  3. ,  1. ,  2. ,
       11. , 12. , 15. , 14. , 13. ,  7.5,  6.5,  0. , 17. , 16. , 18. ])

In [61]:
df_n_agevar['D_2_wk'].isnull().sum()

820

In [62]:
df_n_agevar_mv_count

,nan,비해당,모름
ainc,551,NaN,NaN
D_2_wk,531,30336,289
LQ1_mn,531,35947,297
LQ2_mn,531,37549,309
BD2,795,6152,390
BA2_2_4,1484,36632,213
BA2_2_6,1484,34530,222
BP8,795,701,307
BS3_2,1484,30620,215
BS6_2,6897,21228,236


- 나이와 관련없는 변수는 비해당 값을 0으로 볼 수 있음!! 
> 그렇게 바꿀 시 연속형 변수로 들어감

In [63]:
df_n_agevar_mv_count['ratio(nan+999)'] = (df_n_agevar_mv_count['nan'] + df_n_agevar_mv_count['모름']) /len(df)
df_n_agevar_mv_count['실제 결측치 비율'] = (df_n_agevar_mv_count['nan'] + df_n_agevar_mv_count['모름'])/(len(df)- df_n_agevar_mv_count['비해당'])
df_n_agevar_mv_count['description'] = list(df_n_agevar_info['variable description'])
df_n_agevar_mv_count

,nan,비해당,모름,ratio(nan+999),실제 결측치 비율,description
ainc,551,NaN,NaN,NaN,NaN,월평균 가구총소득
D_2_wk,531,30336,289,0.020624,0.087021,최근2주간 불편감일수
LQ1_mn,531,35947,297,0.020825,0.217209,최근 1달간 와병일수
LQ2_mn,531,37549,309,0.021127,0.38009,최근 1달간 결근일수
BD2,795,6152,390,0.029805,0.035261,(만12세이상) 음주 시작 연령
BA2_2_4,1484,36632,213,0.042682,0.542693,(성인) 1년간 음주운전 횟수: 오토바이
BA2_2_6,1484,34530,222,0.042909,0.326257,(성인) 1년간 음주운전 횟수: 자동차
BP8,795,701,307,0.027717,0.028214,하루 평균 수면시간
BS3_2,1484,30620,215,0.042732,0.185907,(성인) 현재흡연자 하루평균 흡연량
BS6_2,6897,21228,236,0.179406,0.384923,(성인) 과거흡연자 흡연기간(월 환산)


In [64]:
df_n_agevar

,ainc,D_2_wk,LQ1_mn,LQ2_mn,BD2,BA2_2_4,BA2_2_6,BP8,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,40.000000,0.0,0.0,0.0,14.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0
1,40.000000,14.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,30.0
2,700.000000,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
3,333.333333,14.0,0.0,0.0,18.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,20.0
4,333.333333,0.0,0.0,0.0,0.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,666.666667,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39755,666.666667,2.0,0.0,0.0,22.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
39756,430.000000,0.0,0.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0
39757,430.000000,7.0,0.0,0.0,20.0,0.0,0.0,6.0,0.0,180.0,15.0,0.0,20.0,0.0,0.0,0.0,4.0,0.0


In [65]:
df_n_agevar['BS6_2'].value_counts()

0.0      26196
240.0      715
120.0      563
360.0      489
180.0      310
         ...  
53.0         1
563.0        1
311.0        1
254.0        1
68.0         1
Name: BS6_2, Length: 323, dtype: int64

In [66]:
df_n_agevar

,ainc,D_2_wk,LQ1_mn,LQ2_mn,BD2,BA2_2_4,BA2_2_6,BP8,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,40.000000,0.0,0.0,0.0,14.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0
1,40.000000,14.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,30.0
2,700.000000,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
3,333.333333,14.0,0.0,0.0,18.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,20.0
4,333.333333,0.0,0.0,0.0,0.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,666.666667,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39755,666.666667,2.0,0.0,0.0,22.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
39756,430.000000,0.0,0.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0
39757,430.000000,7.0,0.0,0.0,20.0,0.0,0.0,6.0,0.0,180.0,15.0,0.0,20.0,0.0,0.0,0.0,4.0,0.0


In [67]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

for col in df_n_agevar.columns:
    median_imputer = SimpleImputer(strategy='median')
    df_n_agevar[col] = median_imputer.fit_transform(df_n_agevar[[col]])
df_n_agevar

,ainc,D_2_wk,LQ1_mn,LQ2_mn,BD2,BA2_2_4,BA2_2_6,BP8,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,40.000000,0.0,0.0,0.0,14.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,40.000000,14.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
2,700.000000,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,333.333333,14.0,0.0,0.0,18.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
4,333.333333,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,666.666667,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39755,666.666667,2.0,0.0,0.0,22.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
39756,430.000000,0.0,0.0,0.0,19.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0
39757,430.000000,7.0,0.0,0.0,20.0,0.0,0.0,6.0,0.0,180.0,15.0,0.0,20.0,0.0,0.0,0.0,4.0,0.0


# 데이터 합치기

In [68]:
df_agevar.columns

Index(['age', 'DI1_ag', 'DI2_ag', 'DI3_ag', 'DI5_ag', 'DI6_ag', 'DM2_ag',
       'DM3_ag', 'DJ2_ag', 'DJ4_ag', 'DF2_ag', 'DL1_ag', 'DN1_ag', 'DE1_ag',
       'DE2_ag', 'DC1_ag', 'DC2_ag', 'DC3_ag', 'DC4_ag', 'DC5_ag', 'DC6_ag',
       'DC11_ag', 'DK8_ag', 'DK9_ag', 'DK4_ag', 'DI9_ya', 'DF1_ya', 'DN6_ya',
       'DJ9_ya', 'BA2_2_2'],
      dtype='object')

In [69]:
df[list(df_agevar.columns)].head()

,age,DI1_ag,DI2_ag,DI3_ag,DI5_ag,DI6_ag,DM2_ag,DM3_ag,DJ2_ag,DJ4_ag,...,DC6_ag,DC11_ag,DK8_ag,DK9_ag,DK4_ag,DI9_ya,DF1_ya,DN6_ya,DJ9_ya,BA2_2_2
0,70.0,888.0,65.0,888.0,888.0,888.0,69.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
1,68.0,888.0,64.0,888.0,888.0,888.0,60.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
2,48.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
3,35.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,888.0,...,888.0,888.0,888.0,888.0,888.0,NaN,NaN,NaN,NaN,888.0
4,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,888.0,...,NaN,NaN,NaN,NaN,NaN,888.0,888.0,888.0,888.0,NaN


In [70]:
df[list(df_agevar.columns)] = df_agevar
df[list(df_agevar.columns)].head()

,age,DI1_ag,DI2_ag,DI3_ag,DI5_ag,DI6_ag,DM2_ag,DM3_ag,DJ2_ag,DJ4_ag,...,DC6_ag,DC11_ag,DK8_ag,DK9_ag,DK4_ag,DI9_ya,DF1_ya,DN6_ya,DJ9_ya,BA2_2_2
0,6,8,5,8,7,8,5,8,8,8,...,5,8,7,7,6,0,0,0,0,8
1,5,8,5,8,7,8,5,8,8,8,...,5,8,7,7,6,0,0,0,0,8
2,3,8,8,8,7,8,8,8,8,8,...,5,8,7,7,6,0,0,0,0,8
3,2,8,8,8,7,8,8,8,8,8,...,5,8,7,7,6,0,0,0,0,8
4,0,4,4,5,4,5,4,4,1,8,...,2,4,1,4,2,1,1,1,1,0


In [71]:
df_n_agevar.columns

Index(['ainc', 'D_2_wk', 'LQ1_mn', 'LQ2_mn', 'BD2', 'BA2_2_4', 'BA2_2_6',
       'BP8', 'BS3_2', 'BS6_2', 'BS6_2_1', 'BS6_2_2', 'BS6_3', 'BS2', 'BS10_2',
       'BS10_3', 'BE3_32', 'BE3_33'],
      dtype='object')

In [72]:
df[list(df_n_agevar.columns)].head()

,ainc,D_2_wk,LQ1_mn,LQ2_mn,BD2,BA2_2_4,BA2_2_6,BP8,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,40.000000,88.0,88.0,88.0,14.0,888.0,888.0,9.0,10.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,1.0,0.0
1,40.000000,14.0,88.0,88.0,888.0,888.0,888.0,7.0,888.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,0.0,30.0
2,700.000000,88.0,88.0,88.0,888.0,888.0,888.0,9.0,888.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,88.0,88.0
3,333.333333,14.0,88.0,88.0,18.0,888.0,888.0,6.0,888.0,888.0,88.0,88.0,888.0,NaN,NaN,NaN,0.0,20.0
4,333.333333,88.0,88.0,88.0,888.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,88.0,88.0,888.0,0.0,40.0


In [73]:
df[list(df_n_agevar.columns)] = df_n_agevar
df[list(df_n_agevar.columns)].head()

,ainc,D_2_wk,LQ1_mn,LQ2_mn,BD2,BA2_2_4,BA2_2_6,BP8,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,40.000000,0.0,0.0,0.0,14.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,40.000000,14.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
2,700.000000,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,333.333333,14.0,0.0,0.0,18.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
4,333.333333,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0


In [74]:
df_s2f.columns #범주형 변수 합치기

Index(['region', 'town_t', 'apt_t', 'sex', 'incm', 'ho_incm', 'incm5',
       'ho_incm5', 'edu', 'occp',
       ...
       'O_DFID', 'O_DMFIP', 'L_OUT_FQ', 'LK_LB_IT', 'LK_LB_EF', 'LK_EDU',
       'LF_CARE', 'LF_SAFE', 'N_DIET', 'N_DIET_WHY'],
      dtype='object', length=272)

In [75]:
df[list(df_s2f.columns)].head()

,region,town_t,apt_t,sex,incm,ho_incm,incm5,ho_incm5,edu,occp,...,O_DFID,O_DMFIP,L_OUT_FQ,LK_LB_IT,LK_LB_EF,LK_EDU,LF_CARE,LF_SAFE,N_DIET,N_DIET_WHY
0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,...,0.0,1.0,3.0,88.0,8.0,2.0,2.0,2.0,2.0,8.0
1,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,7.0,...,0.0,1.0,3.0,88.0,8.0,2.0,2.0,2.0,1.0,2.0
2,1.0,1.0,1.0,2.0,4.0,4.0,5.0,5.0,3.0,7.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,2.0,3.0,3.0,3.0,4.0,3.0,7.0,...,0.0,1.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,8.0
4,1.0,1.0,1.0,1.0,3.0,3.0,4.0,4.0,1.0,NaN,...,0.0,0.0,3.0,88.0,8.0,1.0,2.0,1.0,2.0,8.0


In [76]:
df[list(df_s2f.columns)] = df_s2f
df[list(df_s2f.columns)].head()

,region,town_t,apt_t,sex,incm,ho_incm,incm5,ho_incm5,edu,occp,...,O_DFID,O_DMFIP,L_OUT_FQ,LK_LB_IT,LK_LB_EF,LK_EDU,LF_CARE,LF_SAFE,N_DIET,N_DIET_WHY
0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,NaN,...,1.0,2.0,3.0,-1.0,-1.0,2.0,2.0,2.0,2.0,-1.0
1,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,7.0,...,1.0,2.0,3.0,-1.0,-1.0,2.0,2.0,2.0,1.0,2.0
2,1.0,1.0,1.0,2.0,4.0,4.0,5.0,5.0,3.0,7.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,1.0,2.0,3.0,3.0,3.0,4.0,3.0,7.0,...,1.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,-1.0
4,1.0,1.0,1.0,1.0,3.0,3.0,4.0,4.0,1.0,NaN,...,1.0,1.0,3.0,-1.0,-1.0,1.0,2.0,1.0,2.0,-1.0


In [77]:
lst_categorical = []

for i in df_col_info['data type'].index:
    if df_col_info['data type'][i] == 'category':
        lst_categorical.append(df_col_info['variable'][i])

In [78]:
lst_categorical

['region',
 'town_t',
 'apt_t',
 'sex',
 'incm',
 'ho_incm',
 'incm5',
 'ho_incm5',
 'edu',
 'occp',
 'cfam',
 'genertn',
 'allownc',
 'house',
 'live_t',
 'marri_1',
 'marri_2',
 'tins',
 'npins',
 'D_1_1',
 'D_2_1',
 'DI1_pr',
 'DI1_dg',
 'DI1_pt',
 'DI1_2',
 'DI2_pr',
 'DI2_dg',
 'DI2_pt',
 'DI2_2',
 'DI3_pr',
 'DI3_dg',
 'DI3_pt',
 'DI3_2',
 'DI4_pr',
 'DI4_dg',
 'DI4_pt',
 'DI5_pr',
 'DI5_dg',
 'DI5_pt',
 'DI6_pr',
 'DI6_dg',
 'DI6_pt',
 'DM1_pr',
 'DM1_dg',
 'DM1_pt',
 'DM2_pr',
 'DM2_dg',
 'DM2_pt',
 'DM3_pr',
 'DM3_dg',
 'DM3_pt',
 'DJ2_pr',
 'DJ2_dg',
 'DJ2_pt',
 'DJ4_pr',
 'DJ4_dg',
 'DJ4_pt',
 'DJ4_3',
 'DF2_pr',
 'DF2_dg',
 'DF2_pt',
 'DL1_pr',
 'DL1_dg',
 'DL1_pt',
 'DN1_pr',
 'DN1_dg',
 'DN1_pt',
 'DE1_pr',
 'DE1_dg',
 'DE1_pt',
 'DE1_3',
 'DE1_31',
 'DE1_32',
 'DE1_33',
 'DE1_34',
 'DE2_pr',
 'DE2_dg',
 'DE2_pt',
 'DC1_pr',
 'DC1_dg',
 'DC1_pt',
 'DC2_pr',
 'DC2_dg',
 'DC2_pt',
 'DC3_pr',
 'DC3_dg',
 'DC3_pt',
 'DC4_pr',
 'DC4_dg',
 'DC4_pt',
 'DC5_pr',
 'DC5_dg',
 'DC5_

In [79]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

for col in df.columns:
    #print(col)
    if col in lst_categorical:                                   # 범주형 변수 (카테고리)
        freq_imputer = SimpleImputer(strategy='most_frequent')
        df[col] = freq_imputer.fit_transform(df[[col]])
        le = LabelEncoder()
        df[col] = le.fit_transform(df[[col]])
        label_map = dict(zip(le.classes_, le.transform(le.classes_))) 
        print(col, "의 라벨링 맵 : ", label_map)
        

    else:
        if col not in agevar_list:
            #mean_imputer = SimpleImputer(strategy='mean')
            median_imputer = SimpleImputer(strategy='median')     # 중앙값
            #df[col] = mean_imputer.fit_transform(df[[col]])      # 평균
            df[col] = median_imputer.fit_transform(df[[col]]) 


region 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 7.0: 6, 8.0: 7, 9.0: 8, 10.0: 9, 11.0: 10, 12.0: 11, 13.0: 12, 14.0: 13, 15.0: 14, 16.0: 15}
town_t 의 라벨링 맵 :  {1.0: 0, 2.0: 1}
apt_t 의 라벨링 맵 :  {1.0: 0, 2.0: 1}
sex 의 라벨링 맵 :  {1.0: 0, 2.0: 1}
incm 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3}
ho_incm 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3}
incm5 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4}
ho_incm5 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4}
edu 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3}
occp 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 7.0: 6}
cfam 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5}
genertn 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 7.0: 6}
allownc 의 라벨링 맵 :  {1.0: 0, 2.0: 1}
house 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2}
live_t 의 라벨링 맵 :  {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4}
marri_1 의 라벨링 맵 :  {1.0: 0, 2.0: 1}
marri_2 의 라벨링 맵 :  {-1.0: 0, 1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4}
tins 의 라벨링 맵 :  {1.0: 0

In [80]:
df

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,2008.0,0,0,0,0,6,1,0,1,0,...,7472.249217,4895.631231,881.735572,5273.157673,2.203960,1.561326,0.987078,16.272761,262.812776,14.85
1,2008.0,0,0,0,1,5,1,0,1,0,...,4430.090787,3680.878220,777.813158,4646.511191,2.188609,1.116872,0.729980,11.567262,206.477449,14.05
2,2008.0,0,0,0,1,3,3,3,4,4,...,3928.688345,2684.283446,568.489544,2778.926550,57.433192,1.179597,1.072590,14.152700,78.229656,10.36
3,2008.0,0,0,0,1,2,2,2,2,3,...,2684.503728,3310.676976,1932.710046,10725.673284,144.044389,0.909731,1.101102,9.129266,147.954071,18.52
4,2008.0,0,0,0,0,0,2,2,3,3,...,2796.580638,2984.102868,434.816014,571.273052,166.161773,1.212672,1.752693,8.201845,65.331394,18.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2014.0,15,0,1,0,0,3,3,4,4,...,3928.688345,2684.283446,568.489544,2778.926550,57.433192,1.179597,1.072590,14.152700,78.229656,22.04
39755,2014.0,15,0,1,1,2,3,3,4,4,...,3685.879396,1880.994894,1933.891026,1906.202132,1615.497101,2.497810,1.278473,14.143210,36.145233,16.25
39756,2014.0,15,0,1,0,2,2,2,2,3,...,2043.568097,1147.876072,300.249632,1793.774635,0.983537,0.791706,0.511638,6.020183,27.892272,18.58
39757,2014.0,15,0,1,0,4,2,2,2,3,...,1758.339159,1446.924818,189.142622,1124.184551,1.043302,1.490047,0.384598,8.750881,24.329294,20.63


In [81]:
df_no_scailing = df.copy() # 정규화 전 데이터 저장 (시각화 위해서)
df_no_scailing

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,2008.0,0,0,0,0,6,1,0,1,0,...,7472.249217,4895.631231,881.735572,5273.157673,2.203960,1.561326,0.987078,16.272761,262.812776,14.85
1,2008.0,0,0,0,1,5,1,0,1,0,...,4430.090787,3680.878220,777.813158,4646.511191,2.188609,1.116872,0.729980,11.567262,206.477449,14.05
2,2008.0,0,0,0,1,3,3,3,4,4,...,3928.688345,2684.283446,568.489544,2778.926550,57.433192,1.179597,1.072590,14.152700,78.229656,10.36
3,2008.0,0,0,0,1,2,2,2,2,3,...,2684.503728,3310.676976,1932.710046,10725.673284,144.044389,0.909731,1.101102,9.129266,147.954071,18.52
4,2008.0,0,0,0,0,0,2,2,3,3,...,2796.580638,2984.102868,434.816014,571.273052,166.161773,1.212672,1.752693,8.201845,65.331394,18.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2014.0,15,0,1,0,0,3,3,4,4,...,3928.688345,2684.283446,568.489544,2778.926550,57.433192,1.179597,1.072590,14.152700,78.229656,22.04
39755,2014.0,15,0,1,1,2,3,3,4,4,...,3685.879396,1880.994894,1933.891026,1906.202132,1615.497101,2.497810,1.278473,14.143210,36.145233,16.25
39756,2014.0,15,0,1,0,2,2,2,2,3,...,2043.568097,1147.876072,300.249632,1793.774635,0.983537,0.791706,0.511638,6.020183,27.892272,18.58
39757,2014.0,15,0,1,0,4,2,2,2,3,...,1758.339159,1446.924818,189.142622,1124.184551,1.043302,1.490047,0.384598,8.750881,24.329294,20.63


In [82]:
df_no_scailing['EC_occp'].unique()

array([ 0,  9,  2,  7,  5,  8,  4,  3,  6,  1, 10])

In [83]:
le.classes_

array([-1.,  1.,  2.,  3.])

## 정규화

In [84]:
numeric_var = list(df_col_info[df_col_info['data type'] == 'numeric']['variable']) + n_agevar_list # 연속형 변수로 보는거 다시처리
len(numeric_var)

76

In [85]:
df[numeric_var]

,year,EQ5D,HE_fst,HE_dprg,HE_PLS,HE_mPLS,HE_sbp1,HE_dbp1,HE_sbp2,HE_dbp2,...,BS3_2,BS6_2,BS6_2_1,BS6_2_2,BS6_3,BS2,BS10_2,BS10_3,BE3_32,BE3_33
0,2008.0,0.817,3.0,5.0,17.0,56.0,128.0,76.0,122.0,70.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2008.0,0.723,16.0,5.0,15.0,56.0,138.0,88.0,128.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
2,2008.0,1.000,16.0,5.0,21.0,56.0,140.0,96.0,134.0,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2008.0,1.000,14.0,5.0,13.0,56.0,106.0,72.0,108.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
4,2008.0,1.000,15.0,5.0,19.0,56.0,84.0,40.0,90.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2014.0,1.000,13.0,5.0,20.0,56.0,100.0,76.0,94.0,68.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39755,2014.0,1.000,13.0,5.0,18.0,56.0,110.0,74.0,110.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
39756,2014.0,1.000,11.0,5.0,17.0,56.0,132.0,98.0,130.0,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,30.0
39757,2014.0,1.000,14.0,5.0,16.0,56.0,148.0,100.0,150.0,102.0,...,0.0,180.0,15.0,0.0,20.0,0.0,0.0,0.0,4.0,0.0


In [86]:
from sklearn.preprocessing import StandardScaler

# 예시 데이터 생성
# data = pd.DataFrame({'x1': [1, 2, 3, 4], 'x2': [100, 200, 300, 400]})

# StandardScaler 객체 생성
scaler = StandardScaler()

# 각 열에 대해 StandardScaler 객체 생성
scalers = {col: StandardScaler() for col in numeric_var}

# 각 열에 대해 스케일링 수행
data_scaled = pd.DataFrame({col: scalers[col].fit_transform(df[[col]])[:,0] for col in numeric_var})

# 스케일링된 데이터를 데이터프레임으로 변환
df[numeric_var] = data_scaled


# 결과 확인
df

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,-1.337160,0,0,0,0,6,1,0,1,0,...,1.012888,1.321272,0.096310,0.222994,-0.310686,0.252820,-0.287877,0.038525,1.704659,14.85
1,-1.337160,0,0,0,1,5,1,0,1,0,...,-0.037454,0.501629,-0.003061,0.115658,-0.310734,-0.294634,-0.641509,-0.497944,1.107824,14.05
2,-1.337160,0,0,0,1,3,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,10.36
3,-1.337160,0,0,0,1,2,2,2,2,3,...,-0.640139,0.251839,1.101258,1.156936,0.129821,-0.549778,-0.131039,-0.775898,0.487808,18.52
4,-1.337160,0,0,0,0,0,2,2,3,3,...,-0.601443,0.031486,-0.331037,-0.582375,0.198510,-0.176632,0.765210,-0.881632,-0.387523,18.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2.087408,15,0,1,0,0,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,22.04
39755,2.087408,15,0,1,1,2,3,3,4,4,...,-0.294402,-0.712826,1.102388,-0.353720,4.699642,1.406329,0.112931,-0.204263,-0.696731,16.25
39756,2.087408,15,0,1,0,2,2,2,2,3,...,-0.861430,-1.207491,-0.459710,-0.372977,-0.314476,-0.695156,-0.941833,-1.130361,-0.784165,18.58
39757,2.087408,15,0,1,0,4,2,2,2,3,...,-0.959909,-1.005711,-0.565952,-0.487669,-0.314291,0.165022,-1.116573,-0.819037,-0.821913,20.63


# 비타민 D 범주화

- 결핍상태: 혈중 25(OH)D3 농도 < 12ng/ml
- 불충분상태: 12ng/ml <= 혈중 25(OH)D3 농도 <= 19.9ng/ml
- 충분상태: 20 <= 혈중 25(OH)D3 농도

In [87]:
def vitD2categorical(value):
    '''
    비타민 D 범주화
    
    args
        value: 비타민 D 혈중 농도
        
    return
        target: '결핍' or '불충분' or '충분'
    '''
    if value < 12:
        return '결핍'
    
    elif (value >= 12) & (value < 20):
        return '불충분'
    
    elif value >= 20:
        return '충분'

In [88]:
df['HE_vitD'] = df['HE_vitD'].map(vitD2categorical)
df_no_scailing['HE_vitD'] = df_no_scailing['HE_vitD'].map(vitD2categorical)

In [89]:
df

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,-1.337160,0,0,0,0,6,1,0,1,0,...,1.012888,1.321272,0.096310,0.222994,-0.310686,0.252820,-0.287877,0.038525,1.704659,불충분
1,-1.337160,0,0,0,1,5,1,0,1,0,...,-0.037454,0.501629,-0.003061,0.115658,-0.310734,-0.294634,-0.641509,-0.497944,1.107824,불충분
2,-1.337160,0,0,0,1,3,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,결핍
3,-1.337160,0,0,0,1,2,2,2,2,3,...,-0.640139,0.251839,1.101258,1.156936,0.129821,-0.549778,-0.131039,-0.775898,0.487808,불충분
4,-1.337160,0,0,0,0,0,2,2,3,3,...,-0.601443,0.031486,-0.331037,-0.582375,0.198510,-0.176632,0.765210,-0.881632,-0.387523,불충분
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2.087408,15,0,1,0,0,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,충분
39755,2.087408,15,0,1,1,2,3,3,4,4,...,-0.294402,-0.712826,1.102388,-0.353720,4.699642,1.406329,0.112931,-0.204263,-0.696731,불충분
39756,2.087408,15,0,1,0,2,2,2,2,3,...,-0.861430,-1.207491,-0.459710,-0.372977,-0.314476,-0.695156,-0.941833,-1.130361,-0.784165,불충분
39757,2.087408,15,0,1,0,4,2,2,2,3,...,-0.959909,-1.005711,-0.565952,-0.487669,-0.314291,0.165022,-1.116573,-0.819037,-0.821913,충분


## target 설정

In [90]:
def target(df, by='analysis1'):
    '''
    비타민 D 범주화
    
    args
        df: pd.DataFrame()
        by: {'analysis1', 'analysis2'}, default='analysis1'
        
    return
        result: pd.DataFrame()
    '''
    
    result = df.copy()
    
    if by == 'analysis1':
        dic_str2categorical = {'결핍': 1, '불충분': 0, '충분': 0}
        result['HE_vitD'] = result['HE_vitD'].map(dic_str2categorical)
        
    elif by == 'analysis2':
        dic_str2categorical = {'결핍': 1, '불충분': 1, '충분': 0}
        result['HE_vitD'] = result['HE_vitD'].map(dic_str2categorical)
        
    return result

In [91]:
df_anal1 = target(df, by='analysis1')
df_anal2 = target(df, by='analysis2')

df_no_scailing_1 = target(df_no_scailing, by='analysis1')
df_no_scailing_2 = target(df_no_scailing, by='analysis2')

In [92]:
df_anal1

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,-1.337160,0,0,0,0,6,1,0,1,0,...,1.012888,1.321272,0.096310,0.222994,-0.310686,0.252820,-0.287877,0.038525,1.704659,0
1,-1.337160,0,0,0,1,5,1,0,1,0,...,-0.037454,0.501629,-0.003061,0.115658,-0.310734,-0.294634,-0.641509,-0.497944,1.107824,0
2,-1.337160,0,0,0,1,3,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,1
3,-1.337160,0,0,0,1,2,2,2,2,3,...,-0.640139,0.251839,1.101258,1.156936,0.129821,-0.549778,-0.131039,-0.775898,0.487808,0
4,-1.337160,0,0,0,0,0,2,2,3,3,...,-0.601443,0.031486,-0.331037,-0.582375,0.198510,-0.176632,0.765210,-0.881632,-0.387523,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2.087408,15,0,1,0,0,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,0
39755,2.087408,15,0,1,1,2,3,3,4,4,...,-0.294402,-0.712826,1.102388,-0.353720,4.699642,1.406329,0.112931,-0.204263,-0.696731,0
39756,2.087408,15,0,1,0,2,2,2,2,3,...,-0.861430,-1.207491,-0.459710,-0.372977,-0.314476,-0.695156,-0.941833,-1.130361,-0.784165,0
39757,2.087408,15,0,1,0,4,2,2,2,3,...,-0.959909,-1.005711,-0.565952,-0.487669,-0.314291,0.165022,-1.116573,-0.819037,-0.821913,0


In [93]:
df_anal2

,year,region,town_t,apt_t,sex,age,incm,ho_incm,incm5,ho_incm5,...,N_NA,N_K,N_VA,N_CAROT,N_RETIN,N_B1,N_B2,N_NIAC,N_VITC,HE_vitD
0,-1.337160,0,0,0,0,6,1,0,1,0,...,1.012888,1.321272,0.096310,0.222994,-0.310686,0.252820,-0.287877,0.038525,1.704659,1
1,-1.337160,0,0,0,1,5,1,0,1,0,...,-0.037454,0.501629,-0.003061,0.115658,-0.310734,-0.294634,-0.641509,-0.497944,1.107824,1
2,-1.337160,0,0,0,1,3,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,1
3,-1.337160,0,0,0,1,2,2,2,2,3,...,-0.640139,0.251839,1.101258,1.156936,0.129821,-0.549778,-0.131039,-0.775898,0.487808,1
4,-1.337160,0,0,0,0,0,2,2,3,3,...,-0.601443,0.031486,-0.331037,-0.582375,0.198510,-0.176632,0.765210,-0.881632,-0.387523,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39754,2.087408,15,0,1,0,0,3,3,4,4,...,-0.210569,-0.170814,-0.203218,-0.204234,-0.139163,-0.217373,-0.170257,-0.203181,-0.250875,0
39755,2.087408,15,0,1,1,2,3,3,4,4,...,-0.294402,-0.712826,1.102388,-0.353720,4.699642,1.406329,0.112931,-0.204263,-0.696731,1
39756,2.087408,15,0,1,0,2,2,2,2,3,...,-0.861430,-1.207491,-0.459710,-0.372977,-0.314476,-0.695156,-0.941833,-1.130361,-0.784165,1
39757,2.087408,15,0,1,0,4,2,2,2,3,...,-0.959909,-1.005711,-0.565952,-0.487669,-0.314291,0.165022,-1.116573,-0.819037,-0.821913,0


# Training

In [94]:
def train(dic, X, y, num=1):
    '''
    모델 학습
    
    args
        dic: dictionary
        X: explanatory variable
        y: target variable
        num: {1, 2}, default=1
        
    return
        pickle file
    '''
    
    for name in dic.keys():
        print('model: {}'.format(name))
        
        clf = dic[name]
        clf.fit(X, y)
        
        with open('clf_{}_anal{}_ver1.pickle'.format(name, num), 'wb') as f:
            pickle.dump(clf, f)

In [95]:
clf_rf = RandomForestClassifier(n_estimators=1000, random_state=42,n_jobs = 30)
clf_gb = GradientBoostingClassifier(n_estimators=1000, random_state=42)
clf_xgb = XGBClassifier(n_estimators=1000, random_state=42,n_jobs = 30)
clf_lgb = LGBMClassifier(n_estimators=1000, random_state=42,n_jobs = 30)
clf_cb = CatBoostClassifier(random_state=42,cat_features = lst_categorical)

In [96]:
dic_name2model = {'rf': clf_rf, 'gb': clf_gb, 'xgb': clf_xgb, 'lgb': clf_lgb, 'cb': clf_cb}

## analysis 2

In [97]:
X2 = df_anal2.iloc[:,:-1]
y2 = df_anal2.iloc[:,-1]

In [98]:
train(dic=dic_name2model, num=2, X=X2, y=y2)

model: rf
model: gb
model: xgb
[19:29:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
model: lgb
model: cb
Learning rate set to 0.049647
0:	learn: 0.6819588	total: 501ms	remaining: 8m 20s
1:	learn: 0.6720567	total: 963ms	remaining: 8m
2:	learn: 0.6632527	total: 1.36s	remaining: 7m 33s
3:	learn: 0.6553019	total: 1.76s	remaining: 7m 18s
4:	learn: 0.6478357	total: 2.25s	remaining: 7m 27s
5:	learn: 0.6415624	total: 2.66s	remaining: 7m 20s
6:	learn: 0.6358422	total: 2.98s	remaining: 7m 3s
7:	learn: 0.6303656	total: 3.39s	remaining: 7m
8:	learn: 0.6253644	total: 3.8s	remaining: 6m 58s
9:	learn: 0.6205023	total: 4.14s	remaining: 6m 49s
10:	learn: 0.6166215	total: 4.54s	remaining: 6m 48s
11:	learn: 0.6127142	total: 4.95s	remaining: 6m 47s
12:	learn: 0.6095351	total: 5.36s	remaining: 6m 47s
13:	learn:

In [101]:
with open('df_anal2.pickle', 'wb') as f:
            pickle.dump(df_anal2, f)
with open('df_no_scailing_2.pickle', 'wb') as f:
            pickle.dump(df_no_scailing_2, f)